In [14]:
!pip install fpdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40714 sha256=46bd955227bf3e6c64f90ec7d87bd7fd979a259733bca9cba6b99eb6f5f00cee
  Stored in directory: c:\users\amit kamthane\appdata\local\pip\cache\wheels\f9\95\ba\f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [1]:
# !pip install scikit-learn
!pip install openai==0.28

In [2]:
!pip install openai upgrade

ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [15]:
import pandas as pd
import numpy as np
from fpdf import FPDF
import os
from dotenv import load_dotenv
from sklearn.ensemble import IsolationForest
import openai

In [4]:
# Set your OpenAI API key
load_dotenv()
openai_api_key = os.environ['OPENAI_API_KEY']

In [5]:
# Sample environmental data (e.g., emissions over time)
data = pd.DataFrame({
    'date': pd.date_range(start='2024-01-01', periods=30, freq='D'),
    'CO2_emissions': np.random.normal(loc=1500, scale=100, size=30),  # Emissions in metric tons
    'NOx_emissions': np.random.normal(loc=300, scale=30, size=30),  # NOx emissions in metric tons
})

# Step 1: Anomaly Detection using Isolation Forest
iso_forest = IsolationForest(contamination=0.05)
data['anomaly'] = iso_forest.fit_predict(data[['CO2_emissions', 'NOx_emissions']])

# Flagging rows that are anomalies
anomalies = data[data['anomaly'] == -1]  

In [6]:
anomalies

,date,CO2_emissions,NOx_emissions,anomaly
0,2024-01-01,1280.196819,286.188987,-1
7,2024-01-08,1729.492271,331.891826,-1


##### Use LLM to generate explanations for anomalies

In [8]:
for _, row in anomalies.iterrows():
    anomaly_description = f"""
    On {row['date'].strftime('%Y-%m-%d')}, an anomaly was detected with CO2 emissions of {row['CO2_emissions']} metric tons and NOx emissions of {row['NOx_emissions']} metric tons.
    These values are outside the expected range based on previous observations.
    """
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=f"Explain the possible causes of this anomaly in environmental emissions data and suggest potential corrective actions:\n\n{anomaly_description}",
        max_tokens=150
    )
    
    # Output the explanation provided by the LLM
    print(response.choices[0].text.strip())

1. Measurement Error: One of the potential causes of this anomaly could be due to measurement error. This could be caused by technical problems, human error, or faulty equipment. It is possible that the emissions data was recorded incorrectly, leading to the outlier values.

Solution: To correct this, the data should be carefully checked for any errors or inconsistencies. If a measurement error is identified, it should be corrected and the emissions data should be updated accordingly.

2. Incorrect Data Entry: Another possible cause of this anomaly could be incorrect data entry. This could happen if there was a mistake made while typing in the values.

Solution: To correct this, the data should be re-checked for any typos or incorrect values. Any errors should be corrected and
Possible Causes:
1. Measurement Error - The anomaly could be due to incorrect measurement of emissions. This could be caused by faulty equipment, human error, or misinterpretation of data.

2. Equipment Malfuncti

##### Automated Reporting and Notifications

In [9]:
# Generate a compliance report for detected anomalies
report = "Environmental Compliance Report\n"
for _, row in anomalies.iterrows():
    report += f"Date: {row['date'].strftime('%Y-%m-%d')}\n"
    report += f"CO2 Emissions: {row['CO2_emissions']} metric tons\n"
    report += f"NOx Emissions: {row['NOx_emissions']} metric tons\n"
    report += f"Anomaly detected. Suggested action: {response.choices[0].text.strip()}\n\n"


In [10]:
report

'Environmental Compliance Report\nDate: 2024-01-01\nCO2 Emissions: 1280.1968194356862 metric tons\nNOx Emissions: 286.18898691866247 metric tons\nAnomaly detected. Suggested action: Possible Causes:\n1. Measurement Error - The anomaly could be due to incorrect measurement of emissions. This could be caused by faulty equipment, human error, or misinterpretation of data.\n\n2. Equipment Malfunction - Malfunctioning equipment or sensors can result in incorrect readings of emissions data. This could be caused by technical issues, lack of maintenance, or outdated equipment.\n\n3. Data Entry Error - The anomaly could be a result of human error while entering the emissions data. This could be caused by typo errors, incorrect conversion units, or incorrect interpretation of data.\n\n4. Unexpected Event - A sudden and unexpected event, such as a natural disaster or industrial accident, could have led to a significant increase in emissions. This could result in abnormal readings\n\nDate: 2024-01

In [17]:
def create_pdf(summary, table_data=None):
    pdf = FPDF()
    pdf.add_page()
    
    # Add title
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Summary", ln=True, align='C')
    
    # Add summary text
    pdf.set_font("Arial", size=10)
    pdf.multi_cell(0, 10, summary)

    # Add table if data is provided
    if table_data:
        pdf.ln(10)
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt="Table", ln=True, align='C')
        
        pdf.set_font("Arial", size=10)
        col_width = pdf.w / 4.5
        row_height = pdf.font_size
        
        for row in table_data:
            for item in row:
                pdf.cell(col_width, row_height*1.5, txt=item, border=1)
            pdf.ln(row_height*1.5)
    
    pdf.output("summary.pdf")



summary = create_pdf(report)